# ESPNet TTSモデル学習

ESPNetを使用して、3つの異なるコーパス条件でTTSモデルを学習します。

## 学習条件

1. **full100**: jvs002 parallel100の全100文
2. **phone_min4**: 音素カバレッジ最小4文（054, 011, 022, 015）
3. **feat_top10**: 特徴量密度トップ10文

## 目的

- ESPNetのVITSモデルを使用
- JSUTで事前学習済みモデルからFine-tuning
- 各条件で同じ設定・同じエポック数で学習
- 学習後、共通の評価文で音声を生成して `outputs/espnet/condition_name/` に保存
- 06_evaluation.ipynbで品質比較（MOS、MCD、F0誤差）を行うための準備

## 参考記事

- [ESPNetで日本語TTSモデルを作る方法](https://qiita.com/niwatiki/items/c2fca0307c8718bf3c61)
- [ツクヨミちゃんの声でESPNet-VITSをFine-tuning](https://qiita.com/RRR_troisR/items/6288b9bdc6e725aa8440)


In [1]:
import sys
from pathlib import Path
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import subprocess
import shutil

# プロジェクトルートを取得
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA_ROOT = PROJECT_ROOT / "data" / "jvs_ver1" / "jvs_ver1"
OUTPUT_ROOT = PROJECT_ROOT / "outputs"

SPEAKER_ID = "jvs002"
ESPNET_ROOT = PROJECT_ROOT / "espnet"
ESPNET_DATA_ROOT = PROJECT_ROOT / "espnet_data"

# パス設定
sys.path.append(str(PROJECT_ROOT / "src"))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_ROOT:", DATA_ROOT)
print("OUTPUT_ROOT:", OUTPUT_ROOT)
print("ESPNET_ROOT:", ESPNET_ROOT)
print("ESPNET_DATA_ROOT:", ESPNET_DATA_ROOT)


PROJECT_ROOT: /mnt/c/dev/minimal-feature-corpus-tts
DATA_ROOT: /mnt/c/dev/minimal-feature-corpus-tts/data/jvs_ver1/jvs_ver1
OUTPUT_ROOT: /mnt/c/dev/minimal-feature-corpus-tts/outputs
ESPNET_ROOT: /mnt/c/dev/minimal-feature-corpus-tts/espnet
ESPNET_DATA_ROOT: /mnt/c/dev/minimal-feature-corpus-tts/espnet_data


In [2]:
# 1. データセット定義の読み込み

# 03の解析結果から音素最小4文を取得
corpus_summary_path = OUTPUT_ROOT / "corpus_analysis" / SPEAKER_ID / "summary.json"
with open(corpus_summary_path, 'r', encoding='utf-8') as f:
    corpus_summary = json.load(f)

phone_min4_ids = corpus_summary['selected_ids']
print("phone_min4 (音素最小4文):")
for i, sid in enumerate(phone_min4_ids, 1):
    print(f"  {i}. {sid}")

# 04の解析結果から特徴量トップ10文を取得
feat_summary_path = OUTPUT_ROOT / "feature_density" / SPEAKER_ID / "summary.json"
with open(feat_summary_path, 'r', encoding='utf-8') as f:
    feat_summary = json.load(f)

feat_top10_ids = feat_summary['selected_ids']
print(f"\nfeat_top10 (特徴量密度トップ10文):")
for i, sid in enumerate(feat_top10_ids, 1):
    print(f"  {i}. {sid}")

# full100は全100文
# 転写テキストから全IDを取得
transcript_path = DATA_ROOT / SPEAKER_ID / "parallel100" / "transcripts_utf8.txt"
all_ids = []
with open(transcript_path, 'r', encoding='utf-8') as f:
    for line in f:
        if ":" in line:
            file_id = line.strip().split(":")[0]
            all_ids.append(file_id)

full100_ids = sorted(all_ids)
print(f"\nfull100 (全100文): {len(full100_ids)}文")

# データセット定義を保存
datasets = {
    "full100": full100_ids,
    "phone_min4": phone_min4_ids,
    "feat_top10": feat_top10_ids
}

print("\nデータセット定義完了:")
for name, ids in datasets.items():
    print(f"  {name}: {len(ids)}文")


phone_min4 (音素最小4文):
  1. VOICEACTRESS100_054
  2. VOICEACTRESS100_011
  3. VOICEACTRESS100_022
  4. VOICEACTRESS100_015

feat_top10 (特徴量密度トップ10文):
  1. VOICEACTRESS100_006
  2. VOICEACTRESS100_033
  3. VOICEACTRESS100_011
  4. VOICEACTRESS100_025
  5. VOICEACTRESS100_045
  6. VOICEACTRESS100_013
  7. VOICEACTRESS100_014
  8. VOICEACTRESS100_004
  9. VOICEACTRESS100_017
  10. VOICEACTRESS100_052

full100 (全100文): 100文

データセット定義完了:
  full100: 100文
  phone_min4: 4文
  feat_top10: 10文


## Phase 1: ESPNet環境構築

ESPNetをクローンしてセットアップします。

**注意**: このセルは初回実行時のみ必要です。既にESPNetがインストールされている場合はスキップできます。


In [3]:
# ESPNetのインストール確認とセットアップ

if not ESPNET_ROOT.exists():
    print("ESPNetが存在しません。クローンを開始します...")
    print("このセルを実行する前に、ターミナルで以下を実行してください:")
    print("  cd /mnt/c/dev/minimal-feature-corpus-tts")
    print("  git clone https://github.com/espnet/espnet.git")
    print("  cd espnet/tools")
    print("  ./setup_anaconda.sh venv espnet 3.9")
    print("  source venv/bin/activate")
    print("  make")
    print("  pip install torch torchaudio --index-url https://download.pytorch.org/whl/cu118  # GPU使用時")
else:
    print(f"ESPNetは既に存在します: {ESPNET_ROOT}")
    
    # 環境の確認
    venv_path = ESPNET_ROOT / "tools" / "venv"
    if venv_path.exists():
        print(f"仮想環境が見つかりました: {venv_path}")
        print("\n仮想環境を有効化するには:")
        print(f"  source {venv_path}/bin/activate")
    else:
        print("仮想環境が見つかりません。セットアップが必要です。")


ESPNetは既に存在します: /mnt/c/dev/minimal-feature-corpus-tts/espnet
仮想環境が見つかりました: /mnt/c/dev/minimal-feature-corpus-tts/espnet/tools/venv

仮想環境を有効化するには:
  source /mnt/c/dev/minimal-feature-corpus-tts/espnet/tools/venv/bin/activate


## Phase 2: ESPNet形式へのデータセット変換

JVSコーパスをESPNetのRecipeで使用できる形式に変換します。


In [4]:
# ESPNet用データセット変換関数

def create_espnet_dataset(dataset_name: str, file_ids: List[str], speaker_id: str = "jvs002") -> Path:
    """
    ESPNet用のデータセットディレクトリを作成
    
    構造:
    espnet_data/{dataset_name}/
    ├── voice_text.txt  # JVS002001:テキスト形式
    └── voice/
        ├── JVS002_001.wav
        ├── JVS002_002.wav
        ...
    
    Args:
        dataset_name: データセット名 (full100, phone_min4, feat_top10)
        file_ids: ファイルIDのリスト
        speaker_id: 話者ID
    
    Returns:
        作成されたデータセットディレクトリのパス
    """
    output_dir = ESPNET_DATA_ROOT / dataset_name
    output_dir.mkdir(parents=True, exist_ok=True)
    
    voice_dir = output_dir / "voice"
    voice_dir.mkdir(exist_ok=True)
    
    # 転写テキストを読み込む
    transcript_path = DATA_ROOT / speaker_id / "parallel100" / "transcripts_utf8.txt"
    transcripts = {}
    with open(transcript_path, 'r', encoding='utf-8') as f:
        for line in f:
            if ':' in line:
                file_id, text = line.strip().split(':', 1)
                transcripts[file_id] = text.strip()
    
    # voice_text.txtを作成
    text_file = output_dir / "voice_text.txt"
    with open(text_file, 'w', encoding='utf-8') as f:
        for i, file_id in enumerate(file_ids, 1):
            # ESPNet形式: JVS002001:テキスト
            espnet_id = f"{speaker_id.upper()}{i:03d}"
            text = transcripts.get(file_id, "")
            if text:
                f.write(f"{espnet_id}:{text}\n")
            
            # 音声ファイルをコピー（名前変更）
            src_wav = DATA_ROOT / speaker_id / "parallel100" / "wav24kHz16bit" / f"{file_id}.wav"
            dst_wav = voice_dir / f"{speaker_id.upper()}_{i:03d}.wav"
            if src_wav.exists():
                shutil.copy(src_wav, dst_wav)
                if (i - 1) % 10 == 0:
                    print(f"  {i}/{len(file_ids)}: {file_id} -> {dst_wav.name}")
            else:
                print(f"  警告: {src_wav} が見つかりません")
    
    print(f"\n✓ ESPNetデータセット作成完了: {output_dir}")
    print(f"  - テキストファイル: {text_file}")
    print(f"  - 音声ファイル数: {len(list(voice_dir.glob('*.wav')))}")
    
    return output_dir

print("ESPNetデータセット変換関数を定義しました。")


ESPNetデータセット変換関数を定義しました。


In [5]:
# 各データセットをESPNet形式に変換

espnet_datasets = {}

for name, ids in datasets.items():
    print(f"\n{'='*60}")
    print(f"データセット変換: {name} ({len(ids)}文)")
    print(f"{'='*60}")
    espnet_datasets[name] = create_espnet_dataset(name, ids, SPEAKER_ID)

print("\n" + "="*60)
print("全データセット変換完了")
print("="*60)
for name, path in espnet_datasets.items():
    print(f"  {name}: {path}")



データセット変換: full100 (100文)
  1/100: VOICEACTRESS100_001 -> JVS002_001.wav
  11/100: VOICEACTRESS100_011 -> JVS002_011.wav
  21/100: VOICEACTRESS100_021 -> JVS002_021.wav
  31/100: VOICEACTRESS100_031 -> JVS002_031.wav
  41/100: VOICEACTRESS100_041 -> JVS002_041.wav
  51/100: VOICEACTRESS100_051 -> JVS002_051.wav
  61/100: VOICEACTRESS100_061 -> JVS002_061.wav
  71/100: VOICEACTRESS100_071 -> JVS002_071.wav
  81/100: VOICEACTRESS100_081 -> JVS002_081.wav
  91/100: VOICEACTRESS100_091 -> JVS002_091.wav

✓ ESPNetデータセット作成完了: /mnt/c/dev/minimal-feature-corpus-tts/espnet_data/full100
  - テキストファイル: /mnt/c/dev/minimal-feature-corpus-tts/espnet_data/full100/voice_text.txt
  - 音声ファイル数: 100

データセット変換: phone_min4 (4文)
  1/4: VOICEACTRESS100_054 -> JVS002_001.wav

✓ ESPNetデータセット作成完了: /mnt/c/dev/minimal-feature-corpus-tts/espnet_data/phone_min4
  - テキストファイル: /mnt/c/dev/minimal-feature-corpus-tts/espnet_data/phone_min4/voice_text.txt
  - 音声ファイル数: 4

データセット変換: feat_top10 (10文)
  1/10: VOICEACTRESS100_0

## Phase 3: ESPNet Recipeセットアップ

各データセット条件ごとにESPNetのRecipeを作成します。


In [6]:
# Recipeディレクトリの作成とセットアップ

def setup_recipe(dataset_name: str, espnet_data_dir: Path) -> Path:
    """
    ESPNetのRecipeをセットアップ
    
    Args:
        dataset_name: データセット名
        espnet_data_dir: ESPNet形式データのディレクトリ
    
    Returns:
        Recipeディレクトリのパス
        
    Raises:
        FileNotFoundError: ESPNetがインストールされていない場合
    """
    # ESPNetがインストールされているか確認
    if not ESPNET_ROOT.exists():
        raise FileNotFoundError(
            f"\n{'='*60}\n"
            f"❌ ESPNetがインストールされていません！\n"
            f"{'='*60}\n\n"
            f"ESPNetディレクトリが見つかりません: {ESPNET_ROOT}\n\n"
            f"📦 ESPNetをインストールするには、ターミナルで以下を実行してください:\n\n"
            f"  cd {PROJECT_ROOT}\n"
            f"  git clone https://github.com/espnet/espnet.git\n"
            f"  cd espnet/tools\n"
            f"  ./setup_anaconda.sh venv espnet 3.9\n"
            f"  source venv/bin/activate\n"
            f"  make\n"
            f"  pip install torch torchaudio --index-url https://download.pytorch.org/whl/cu118  # GPU使用時\n\n"
            f"詳細はセル4（Phase 1: ESPNet環境構築）を参照してください。\n"
            f"{'='*60}"
        )
    
    recipe_dir = ESPNET_ROOT / "egs2" / f"{SPEAKER_ID}_{dataset_name}" / "tts1"
    
    # Recipeが存在しない場合はテンプレートから作成
    if not recipe_dir.exists():
        print(f"Recipeを作成中: {recipe_dir}")
        template_dir = ESPNET_ROOT / "egs2" / "TEMPLATE" / "tts1"
        
        if not template_dir.exists():
            raise FileNotFoundError(
                f"\n{'='*60}\n"
                f"❌ ESPNetテンプレートが見つかりません！\n"
                f"{'='*60}\n\n"
                f"テンプレートディレクトリ: {template_dir}\n"
                f"ESPNetディレクトリ: {ESPNET_ROOT}\n\n"
                f"ESPNetが正しくインストールされていない可能性があります。\n"
                f"ESPNetのセットアップを完了してください（セル4を参照）。\n"
                f"{'='*60}"
            )
        
        # setup.shを実行
        import os
        os.chdir(template_dir)
        result = subprocess.run(
            ["./setup.sh", str(recipe_dir)],
            capture_output=True,
            text=True
        )
        
        if result.returncode != 0:
            print(f"エラー: {result.stderr}")
            raise RuntimeError(f"Recipeのセットアップに失敗しました")
        
        print(f"✓ Recipe作成完了: {recipe_dir}")
    else:
        print(f"Recipeは既に存在します: {recipe_dir}")
    
    return recipe_dir

print("Recipeセットアップ関数を定義しました。")


Recipeセットアップ関数を定義しました。


In [7]:
# db.shファイルの作成（データセットパスを定義）

def create_db_sh(recipe_dir: Path, espnet_data_dir: Path):
    """db.shファイルを作成/更新"""
    db_sh_path = recipe_dir / "db.sh"
    
    # データセット変数を追加
    content = f"""# JVS002 {espnet_data_dir.name} dataset
JVS002_{espnet_data_dir.name.upper()}={espnet_data_dir}
"""
    
    # 既存のdb.shがある場合は追記、ない場合は新規作成
    if db_sh_path.exists():
        with open(db_sh_path, 'r', encoding='utf-8') as f:
            existing = f.read()
        if espnet_data_dir.name not in existing:
            with open(db_sh_path, 'a', encoding='utf-8') as f:
                f.write(content)
            print(f"✓ db.shに追加: {db_sh_path}")
        else:
            print(f"db.shには既に追加済みです: {db_sh_path}")
    else:
        with open(db_sh_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"✓ db.shを作成: {db_sh_path}")
    
    return db_sh_path

print("db.sh作成関数を定義しました。")


db.sh作成関数を定義しました。


In [8]:
# local/data.sh と local/data_prep.sh の作成

def create_data_scripts(recipe_dir: Path, dataset_name: str, espnet_data_dir: Path):
    """データ準備用スクリプトを作成"""
    local_dir = recipe_dir / "local"
    local_dir.mkdir(exist_ok=True)
    
    # data.sh: データセットパスの設定
    data_sh_path = local_dir / "data.sh"
    data_sh_content = f"""#!/bin/bash
# Data paths for {dataset_name}

# ESPNet data directory
data_dir={espnet_data_dir}

# Output directory for processed data
dumpdir=dump/24k

# Speaker ID
spk=jvs002

# Dataset name
dset={dataset_name}
"""
    
    with open(data_sh_path, 'w', encoding='utf-8') as f:
        f.write(data_sh_content)
    data_sh_path.chmod(0o755)
    print(f"✓ local/data.shを作成: {data_sh_path}")
    
    # data_prep.sh: データ準備スクリプト（Qiita記事を参考）
    data_prep_sh_path = local_dir / "data_prep.sh"
    data_prep_content = f"""#!/bin/bash
# Data preparation script for {dataset_name}

set -euo pipefail

. ./path.sh || exit 1;
. ./cmd.sh || exit 1;
. ./db.sh || exit 1;

data_dir={espnet_data_dir}
dumpdir=dump/24k

train_set="tr_no_dev"
dev_set="dev"
eval_set="eval"

# Text file path
text_file="${{data_dir}}/voice_text.txt"
wav_dir="${{data_dir}}/voice"

# Create output directories
mkdir -p "${{dumpdir}}"
train_dumpdir="${{dumpdir}}/${{train_set}}"
dev_dumpdir="${{dumpdir}}/${{dev_set}}"
eval_dumpdir="${{dumpdir}}/${{eval_set}}"
mkdir -p "${{train_dumpdir}}" "${{dev_dumpdir}}" "${{eval_dumpdir}}"

# Split data (80% train, 10% dev, 10% eval)
total_lines=$(wc -l < "${{text_file}}")
train_lines=$((total_lines * 8 / 10))
dev_lines=$((total_lines * 9 / 10))

head -n "${{train_lines}}" "${{text_file}}" > "${{dumpdir}}/train_text.txt"
sed -n "$((train_lines+1)),${{dev_lines}}p" "${{text_file}}" > "${{dumpdir}}/dev_text.txt"
tail -n +$((dev_lines+1)) "${{text_file}}" > "${{dumpdir}}/eval_text.txt"

echo "Data preparation completed."
echo "Train: ${{train_lines}} lines"
echo "Dev: $((dev_lines - train_lines)) lines"
echo "Eval: $((total_lines - dev_lines)) lines"
"""
    
    with open(data_prep_sh_path, 'w', encoding='utf-8') as f:
        f.write(data_prep_content)
    data_prep_sh_path.chmod(0o755)
    print(f"✓ local/data_prep.shを作成: {data_prep_sh_path}")
    
    return data_sh_path, data_prep_sh_path

print("データ準備スクリプト作成関数を定義しました。")


データ準備スクリプト作成関数を定義しました。


In [9]:
# 全データセット条件のRecipeをセットアップ

recipe_dirs = {}

# ESPNetがインストールされているか事前に確認
if not ESPNET_ROOT.exists():
    print(f"\n{'='*60}")
    print("⚠️  ESPNetがインストールされていません")
    print(f"{'='*60}")
    print(f"\nESPNetディレクトリが見つかりません: {ESPNET_ROOT}")
    print("\nこのセルを実行する前に、以下の手順を実行してください:")
    print("\n1. セル4（Phase 1: ESPNet環境構築）を確認")
    print("2. ターミナルでESPNetをインストール:")
    print(f"   cd {PROJECT_ROOT}")
    print("   git clone https://github.com/espnet/espnet.git")
    print("   cd espnet/tools")
    print("   ./setup_anaconda.sh venv espnet 3.9")
    print("   source venv/bin/activate")
    print("   make")
    print("\nESPNetをインストールした後、このセルを再実行してください。")
    print(f"{'='*60}\n")
else:
    for dataset_name, espnet_data_dir in espnet_datasets.items():
        print(f"\n{'='*60}")
        print(f"Recipeセットアップ: {dataset_name}")
        print(f"{'='*60}")
        
        try:
            # Recipe作成
            recipe_dir = setup_recipe(dataset_name, espnet_data_dir)
            recipe_dirs[dataset_name] = recipe_dir
            
            # db.sh作成
            create_db_sh(recipe_dir, espnet_data_dir)
            
            # データ準備スクリプト作成
            create_data_scripts(recipe_dir, dataset_name, espnet_data_dir)
        except FileNotFoundError as e:
            print(f"\n❌ エラーが発生しました:")
            print(str(e))
            print(f"\n{dataset_name}のRecipeセットアップをスキップします。")
            continue
        except Exception as e:
            print(f"\n❌ 予期しないエラー: {e}")
            print(f"{dataset_name}のRecipeセットアップをスキップします。")
            continue

    if recipe_dirs:
        print("\n" + "="*60)
        print("Recipeセットアップ完了")
        print("="*60)
        for name, path in recipe_dirs.items():
            print(f"  {name}: {path}")
    else:
        print("\n⚠️  すべてのRecipeセットアップがスキップされました。")
        print("ESPNetをインストールしてから再実行してください。")



Recipeセットアップ: full100
Recipeは既に存在します: /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1
db.shには既に追加済みです: /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1/db.sh
✓ local/data.shを作成: /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1/local/data.sh
✓ local/data_prep.shを作成: /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1/local/data_prep.sh

Recipeセットアップ: phone_min4
Recipeは既に存在します: /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_phone_min4/tts1
db.shには既に追加済みです: /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_phone_min4/tts1/db.sh
✓ local/data.shを作成: /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_phone_min4/tts1/local/data.sh
✓ local/data_prep.shを作成: /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_phone_min4/tts1/local/data_prep.sh

Recipeセットアップ: feat_top10
Recipeは既に存在します: /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_feat_top10/tts1
db.shには既に追加済みです: /mnt/c/dev/mi

## Phase 4: データ準備（Stage 1-5）

ESPNetのデータ準備ステージを実行します。この段階では特徴量抽出、トークン化などが行われます。

**注意**: このセルはESPNetの仮想環境を有効化した状態で実行する必要があります。


In [10]:
# データ準備実行用のコマンド生成

def generate_data_prep_command(recipe_dir: Path, dataset_name: str) -> str:
    """
    データ準備（Stage 1-5）のコマンドを生成
    
    Qiita記事の設定を参考:
    - g2p: pyopenjtalk_accent_with_pause
    - fs: 24000
    - n_fft: 1024
    - n_shift: 256
    - tts_task: gan_tts (VITS)
    """
    cmd = f"""cd {recipe_dir}
./run.sh \\
  --stage 1 \\
  --stop-stage 5 \\
  --g2p pyopenjtalk_accent_with_pause \\
  --min_wav_duration 0.38 \\
  --fs 24000 \\
  --n_fft 1024 \\
  --n_shift 256 \\
  --dumpdir dump/24k \\
  --win_length null \\
  --tts_task gan_tts \\
  --feats_extract linear_spectrogram \\
  --feats_normalize none \\
  --train_config ./conf/tuning/finetune_vits.yaml \\
  --nj 4
"""
    return cmd

# 各データセット条件のコマンドを生成
print("データ準備コマンド（Stage 1-5）:")
print("="*60)
for dataset_name, recipe_dir in recipe_dirs.items():
    cmd = generate_data_prep_command(recipe_dir, dataset_name)
    print(f"\n【{dataset_name}】")
    print(cmd)
    print("-"*60)


データ準備コマンド（Stage 1-5）:

【full100】
cd /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1
./run.sh \
  --stage 1 \
  --stop-stage 5 \
  --g2p pyopenjtalk_accent_with_pause \
  --min_wav_duration 0.38 \
  --fs 24000 \
  --n_fft 1024 \
  --n_shift 256 \
  --dumpdir dump/24k \
  --win_length null \
  --tts_task gan_tts \
  --feats_extract linear_spectrogram \
  --feats_normalize none \
  --train_config ./conf/tuning/finetune_vits.yaml \
  --nj 4

------------------------------------------------------------

【phone_min4】
cd /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_phone_min4/tts1
./run.sh \
  --stage 1 \
  --stop-stage 5 \
  --g2p pyopenjtalk_accent_with_pause \
  --min_wav_duration 0.38 \
  --fs 24000 \
  --n_fft 1024 \
  --n_shift 256 \
  --dumpdir dump/24k \
  --win_length null \
  --tts_task gan_tts \
  --feats_extract linear_spectrogram \
  --feats_normalize none \
  --train_config ./conf/tuning/finetune_vits.yaml \
  --nj 4

---------------------------

## Phase 5: Fine-tuning設定（JSUT事前学習モデル）

JSUTで事前学習済みのVITSモデルをダウンロードして、Fine-tuningの準備をします。


In [11]:
# JSUT事前学習モデルのダウンロードと設定用コマンド

def generate_finetune_setup_commands(recipe_dir: Path) -> List[str]:
    """
    Fine-tuning設定用のコマンドリストを生成
    
    1. JSUT事前学習モデルのダウンロード
    2. トークンリストの準備
    3. シンボリックリンクの作成
    """
    commands = []
    
    # 1. モデルダウンロード
    download_cmd = f"""cd {recipe_dir}
source ../../../tools/venv/bin/activate
espnet_model_zoo_download --unpack true --cachedir downloads \\
  kan-bayashi/jsut_vits_accent_with_pause
"""
    commands.append(("モデルダウンロード", download_cmd))
    
    # 2. トークンリスト準備
    token_cmd = f"""cd {recipe_dir}
source ../../../tools/venv/bin/activate
pyscripts/utils/make_token_list_from_config.py \\
  downloads/*/exp/tts_train_vits_raw_phn_jaconv_pyopenjtalk_accent_with_pause/config.yaml
"""
    commands.append(("トークンリスト準備", token_cmd))
    
    # 3. シンボリックリンク作成
    link_cmd = f"""cd {recipe_dir}
mv dump/24k/token_list/phn_jaconv_pyopenjtalk_accent_with_pause/tokens.txt \\
  dump/24k/token_list/phn_jaconv_pyopenjtalk_accent_with_pause/tokens.txt.bak 2>/dev/null || true
ln -s $(pwd)/downloads/*/exp/tts_train_vits_raw_phn_jaconv_pyopenjtalk_accent_with_pause/tokens.txt \\
  dump/24k/token_list/phn_jaconv_pyopenjtalk_accent_with_pause/tokens.txt
"""
    commands.append(("シンボリックリンク作成", link_cmd))
    
    return commands

print("Fine-tuning設定コマンド:")
print("="*60)

for dataset_name, recipe_dir in recipe_dirs.items():
    print(f"\n【{dataset_name}】")
    print("-"*60)
    commands = generate_finetune_setup_commands(recipe_dir)
    for step_name, cmd in commands:
        print(f"\n--- {step_name} ---")
        print(cmd)


Fine-tuning設定コマンド:

【full100】
------------------------------------------------------------

--- モデルダウンロード ---
cd /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1
source ../../../tools/venv/bin/activate
espnet_model_zoo_download --unpack true --cachedir downloads \
  kan-bayashi/jsut_vits_accent_with_pause


--- トークンリスト準備 ---
cd /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1
source ../../../tools/venv/bin/activate
pyscripts/utils/make_token_list_from_config.py \
  downloads/*/exp/tts_train_vits_raw_phn_jaconv_pyopenjtalk_accent_with_pause/config.yaml


--- シンボリックリンク作成 ---
cd /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1
mv dump/24k/token_list/phn_jaconv_pyopenjtalk_accent_with_pause/tokens.txt \
  dump/24k/token_list/phn_jaconv_pyopenjtalk_accent_with_pause/tokens.txt.bak 2>/dev/null || true
ln -s $(pwd)/downloads/*/exp/tts_train_vits_raw_phn_jaconv_pyopenjtalk_accent_with_pause/tokens.txt \
  dump/24k/token_list/phn_ja

## Phase 6: 学習実行（Stage 6）

Fine-tuning学習を実行します。この段階は時間がかかります（数時間〜数十時間）。

**実行前に確認:**
- データ準備（Stage 1-5）が完了していること
- JSUT事前学習モデルのダウンロードが完了していること
- トークンリストの設定が完了していること


In [12]:
# 学習実行（Stage 6）用のコマンド生成

def generate_training_command(recipe_dir: Path, dataset_name: str, tag_suffix: str = "") -> str:
    """
    Fine-tuning学習（Stage 6）のコマンドを生成
    
    Args:
        recipe_dir: Recipeディレクトリのパス
        dataset_name: データセット名
        tag_suffix: タグのサフィックス（オプション）
    """
    tag = f"finetune_vits_{SPEAKER_ID}_{dataset_name}"
    if tag_suffix:
        tag += f"_{tag_suffix}"
    
    cmd = f"""cd {recipe_dir}
./run.sh \\
  --stage 6 \\
  --g2p pyopenjtalk_accent_with_pause \\
  --min_wav_duration 0.38 \\
  --fs 24000 \\
  --n_fft 1024 \\
  --n_shift 256 \\
  --dumpdir dump/24k \\
  --win_length null \\
  --tts_task gan_tts \\
  --feats_extract linear_spectrogram \\
  --feats_normalize none \\
  --train_config ./conf/tuning/finetune_vits.yaml \\
  --train_args "--init_param downloads/*/exp/tts_train_vits_raw_phn_jaconv_pyopenjtalk_accent_with_pause/train.total_count.ave_10best.pth:tts:tts" \\
  --tag {tag}
"""
    return cmd

print("学習実行コマンド（Stage 6）:")
print("="*60)
for dataset_name, recipe_dir in recipe_dirs.items():
    cmd = generate_training_command(recipe_dir, dataset_name)
    print(f"\n【{dataset_name}】")
    print(cmd)
    print("-"*60)


学習実行コマンド（Stage 6）:

【full100】
cd /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1
./run.sh \
  --stage 6 \
  --g2p pyopenjtalk_accent_with_pause \
  --min_wav_duration 0.38 \
  --fs 24000 \
  --n_fft 1024 \
  --n_shift 256 \
  --dumpdir dump/24k \
  --win_length null \
  --tts_task gan_tts \
  --feats_extract linear_spectrogram \
  --feats_normalize none \
  --train_config ./conf/tuning/finetune_vits.yaml \
  --train_args "--init_param downloads/*/exp/tts_train_vits_raw_phn_jaconv_pyopenjtalk_accent_with_pause/train.total_count.ave_10best.pth:tts:tts" \
  --tag finetune_vits_jvs002_full100

------------------------------------------------------------

【phone_min4】
cd /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_phone_min4/tts1
./run.sh \
  --stage 6 \
  --g2p pyopenjtalk_accent_with_pause \
  --min_wav_duration 0.38 \
  --fs 24000 \
  --n_fft 1024 \
  --n_shift 256 \
  --dumpdir dump/24k \
  --win_length null \
  --tts_task gan_tts \
  --feats_extract l

## 実行手順まとめ

### 1. 環境構築（初回のみ）

ターミナルで実行:

```bash
cd /mnt/c/dev/minimal-feature-corpus-tts
git clone https://github.com/espnet/espnet.git
cd espnet/tools
./setup_anaconda.sh venv espnet 3.9
source venv/bin/activate
make
# GPU使用時
pip install torch torchaudio --index-url https://download.pytorch.org/whl/cu118
```

### 2. ノートブック実行

このノートブックを順番に実行:
- データセット変換（セル7まで）
- Recipeセットアップ（セル12まで）

### 3. データ準備（各データセット条件ごと）

ターミナルで実行（ESPNet環境を有効化後）:

```bash
cd /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1
source ../../../tools/venv/bin/activate
# セル14で生成されたコマンドを実行
```

### 4. Fine-tuning設定（各データセット条件ごと）

```bash
# セル16で生成されたコマンドを実行
```

### 5. 学習実行（各データセット条件ごと）

```bash
# セル18で生成されたコマンドを実行
# バックグラウンド実行推奨: nohup ./run.sh ... > train.log 2>&1 &
```

### 学習時間の目安

- **full100**: 18-22時間（Qiita実績より）
- **phone_min4**: 2-4時間（データ量が少ないため）
- **feat_top10**: 4-6時間

## 次のステップ

学習が完了したら:
1. 学習済みモデルを使用して音声を生成
2. `outputs/espnet/{condition_name}/` に評価用音声を保存
3. `06_evaluation.ipynb` で品質評価を実施


## Phase 7: 音声生成と評価

学習が完了したら、評価用の音声を生成します。


In [13]:
# 評価用テキストの準備（共通評価文）

# 評価用のテキストリストを作成
# 音素カバレッジが高い、多様な音素を含む文を選択
eval_texts = [
    "また、東寺のように、五大明王と呼ばれる、主要な明王の中央に配されることも多い。",
    "ニューイングランド風は、牛乳をベースとした、白いクリームスープであり、ボストンクラムチャウダーとも呼ばれる。",
    "サービスマネージャー導入駅のため、大井町駅から、遠隔管理している。",
    "時間領域と、空間領域で共通する処理手法は、フィルタリングによる、入力信号の強化である。",
    "スマートフォンから、フィーチャーフォンまで、マルチデバイスに対応。",
]

# 評価用テキストを保存
eval_text_path = OUTPUT_ROOT / "espnet" / "eval_texts.txt"
eval_text_path.parent.mkdir(parents=True, exist_ok=True)

with open(eval_text_path, 'w', encoding='utf-8') as f:
    for i, text in enumerate(eval_texts, 1):
        f.write(f"eval_{i:03d}:{text}\n")

print(f"評価用テキストを保存しました: {eval_text_path}")
print(f"\n評価文数: {len(eval_texts)}")
for i, text in enumerate(eval_texts, 1):
    print(f"  {i}. {text[:50]}...")


評価用テキストを保存しました: /mnt/c/dev/minimal-feature-corpus-tts/outputs/espnet/eval_texts.txt

評価文数: 5
  1. また、東寺のように、五大明王と呼ばれる、主要な明王の中央に配されることも多い。...
  2. ニューイングランド風は、牛乳をベースとした、白いクリームスープであり、ボストンクラムチャウダーとも呼...
  3. サービスマネージャー導入駅のため、大井町駅から、遠隔管理している。...
  4. 時間領域と、空間領域で共通する処理手法は、フィルタリングによる、入力信号の強化である。...
  5. スマートフォンから、フィーチャーフォンまで、マルチデバイスに対応。...


In [14]:
# 音声生成用コマンド生成

def generate_inference_command(recipe_dir: Path, dataset_name: str, eval_text_path: Path) -> str:
    """
    学習済みモデルで音声生成を行うコマンドを生成
    
    Args:
        recipe_dir: Recipeディレクトリ
        dataset_name: データセット名
        eval_text_path: 評価用テキストファイルのパス
    """
    # 出力ディレクトリ
    output_dir = OUTPUT_ROOT / "espnet" / dataset_name
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # モデルパス（学習が完了した場合）
    model_path = recipe_dir / "exp" / f"finetune_vits_{SPEAKER_ID}_{dataset_name}" / "train.total_count.ave.pth"
    
    cmd = f"""# 音声生成コマンド（学習完了後）
cd {recipe_dir}
source ../../../tools/venv/bin/activate

# 学習済みモデルで音声生成
espnet2_tts_inference \\
  --train_config exp/finetune_vits_{SPEAKER_ID}_{dataset_name}/config.yaml \\
  --model_file exp/finetune_vits_{SPEAKER_ID}_{dataset_name}/train.total_count.ave.pth \\
  --vocoder_file downloads/*/exp/tts_train_vits_raw_phn_jaconv_pyopenjtalk_accent_with_pause/train.total_count.ave_10best.pth \\
  --inference_args "--use_teacher_forcing true" \\
  --text_list {eval_text_path} \\
  --output_dir {output_dir}

# または、run.shのStage 7を使用
# ./run.sh \\
#   --stage 7 \\
#   --train_config ./conf/tuning/finetune_vits.yaml \\
#   --model_file exp/finetune_vits_{SPEAKER_ID}_{dataset_name}/train.total_count.ave.pth \\
#   --text_list {eval_text_path} \\
#   --output_dir {output_dir}
"""
    return cmd

print("音声生成コマンド:")
print("="*60)
for dataset_name, recipe_dir in recipe_dirs.items():
    cmd = generate_inference_command(recipe_dir, dataset_name, eval_text_path)
    print(f"\n【{dataset_name}】")
    print(cmd)
    print("-"*60)


音声生成コマンド:

【full100】
# 音声生成コマンド（学習完了後）
cd /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1
source ../../../tools/venv/bin/activate

# 学習済みモデルで音声生成
espnet2_tts_inference \
  --train_config exp/finetune_vits_jvs002_full100/config.yaml \
  --model_file exp/finetune_vits_jvs002_full100/train.total_count.ave.pth \
  --vocoder_file downloads/*/exp/tts_train_vits_raw_phn_jaconv_pyopenjtalk_accent_with_pause/train.total_count.ave_10best.pth \
  --inference_args "--use_teacher_forcing true" \
  --text_list /mnt/c/dev/minimal-feature-corpus-tts/outputs/espnet/eval_texts.txt \
  --output_dir /mnt/c/dev/minimal-feature-corpus-tts/outputs/espnet/full100

# または、run.shのStage 7を使用
# ./run.sh \
#   --stage 7 \
#   --train_config ./conf/tuning/finetune_vits.yaml \
#   --model_file exp/finetune_vits_jvs002_full100/train.total_count.ave.pth \
#   --text_list /mnt/c/dev/minimal-feature-corpus-tts/outputs/espnet/eval_texts.txt \
#   --output_dir /mnt/c/dev/minimal-feature-corpus-tts/out

## 学習状況の確認

学習中のログや進捗を確認する方法。

### なぜなぜ分析

**問題**: 学習ディレクトリが存在しない

**なぜなぜ分析**:

1. **なぜ学習ディレクトリが存在しないのか？**
   → 学習が開始されていないから

2. **なぜ学習が開始されていないのか？**
   → 学習コマンド（Phase 6）が実行されていないから

3. **なぜ学習コマンドが実行されていないのか？**
   → ノートブックはコマンドを**生成するだけ**で、実際の実行は**ターミナルで手動実行**する必要があるから

4. **なぜ学習コマンドが実行されていないのか？**
   → 学習前の準備ステップ（データ準備、Fine-tuning設定）が未完了の可能性があるから

**根本原因**:
- ノートブックは学習コマンドを生成するだけで、自動実行は行わない
- 学習前の準備ステップ（Phase 4, 5）が未完了
- 学習状況確認が不十分で、どのステップが未完了か分からない

**解決策**:
- 学習状況確認関数を改善（各ステップの完了状況を詳細に確認）
- 未完了のステップを明確に表示
- 次のアクションを具体的に示す


In [15]:
# 学習状況確認用のヘルパー関数（改善版：なぜなぜ分析対応）

def check_training_status(recipe_dir: Path, dataset_name: str):
    """
    学習状況を詳細に確認（なぜなぜ分析対応）
    
    確認項目:
    1. Recipeディレクトリの存在
    2. データ準備（dumpディレクトリ）の状況
    3. Fine-tuning設定（downloadsディレクトリ）の状況
    4. 学習ディレクトリ（exp）の状況
    5. チェックポイントの状況
    """
    print(f"\n【{dataset_name}】学習状況チェック")
    print("="*60)
    
    # 1. Recipeディレクトリの確認
    if not recipe_dir.exists():
        print(f"❌ Recipeディレクトリが存在しません: {recipe_dir}")
        print("  → セル12（Phase 3: Recipeセットアップ）を実行してください")
        return
    else:
        print(f"✓ Recipeディレクトリ: {recipe_dir}")
    
    # 2. データ準備（Stage 1-5）の確認
    dump_dir = recipe_dir / "dump" / "24k"
    if not dump_dir.exists():
        print(f"❌ データ準備が未実行です（dump/24kディレクトリが見つかりません）")
        print("  → セル14で生成されたコマンドをターミナルで実行してください")
        print("  → Phase 4: データ準備（Stage 1-5）を参照")
    else:
        train_dump = dump_dir / "tr_no_dev"
        if train_dump.exists():
            num_files = len(list(train_dump.glob("*.npy"))) if train_dump.exists() else 0
            print(f"✓ データ準備完了（dump/24k/tr_no_dev に {num_files} ファイル）")
        else:
            print(f"⚠ データ準備が不完全の可能性（tr_no_devディレクトリが見つかりません）")
    
    # 3. Fine-tuning設定の確認
    downloads_dir = recipe_dir / "downloads"
    jsut_model_dir = None
    if downloads_dir.exists():
        # JSUTモデルのディレクトリを探す
        for item in downloads_dir.iterdir():
            if item.is_dir() and "jsut" in item.name.lower():
                exp_dir_check = item / "exp" / "tts_train_vits_raw_phn_jaconv_pyopenjtalk_accent_with_pause"
                if exp_dir_check.exists():
                    jsut_model_dir = exp_dir_check
                    break
        
        if jsut_model_dir:
            model_file = jsut_model_dir / "train.total_count.ave_10best.pth"
            if model_file.exists():
                print(f"✓ JSUT事前学習モデルがダウンロード済み")
            else:
                print(f"⚠ JSUTモデルファイルが見つかりません: {model_file}")
        else:
            print(f"⚠ JSUT事前学習モデルのディレクトリが見つかりません")
            print("  → セル16で生成されたコマンドをターミナルで実行してください")
            print("  → Phase 5: Fine-tuning設定を参照")
    else:
        print(f"❌ Fine-tuning設定が未実行です（downloadsディレクトリが見つかりません）")
        print("  → セル16で生成されたコマンドをターミナルで実行してください")
        print("  → Phase 5: Fine-tuning設定を参照")
    
    # 4. 学習ディレクトリの確認
    exp_dir = recipe_dir / "exp" / f"finetune_vits_{SPEAKER_ID}_{dataset_name}"
    
    if not exp_dir.exists():
        print(f"\n❌ 学習が開始されていません（expディレクトリが存在しません）")
        print("\n📋 学習を開始する前に以下を確認:")
        
        missing_steps = []
        if not dump_dir.exists():
            missing_steps.append("Phase 4: データ準備（Stage 1-5）")
        if not downloads_dir.exists() or not jsut_model_dir:
            missing_steps.append("Phase 5: Fine-tuning設定")
        
        if missing_steps:
            print(f"  ⚠ 以下のステップが未完了です:")
            for step in missing_steps:
                print(f"     - {step}")
            print(f"\n  → まず上記のステップを完了してから、Phase 6: 学習実行（Stage 6）を実行してください")
        else:
            print(f"  ✓ 準備は完了しているようです")
            print(f"  → セル18で生成されたコマンドをターミナルで実行して学習を開始してください")
            print(f"  → Phase 6: 学習実行（Stage 6）を参照")
        return
    
    print(f"✓ 学習ディレクトリ: {exp_dir}")
    
    # 5. チェックポイントファイルの確認
    checkpoint_file = exp_dir / "train.total_count.ave.pth"
    if checkpoint_file.exists():
        file_size = checkpoint_file.stat().st_size / (1024 * 1024)  # MB
        print(f"✓ 学習済みモデルが見つかりました")
        print(f"  モデルパス: {checkpoint_file}")
        print(f"  ファイルサイズ: {file_size:.2f} MB")
    else:
        print("⚠ 学習中の可能性があります")
        # 最新のチェックポイントを探す
        checkpoints = list(exp_dir.glob("*.pth"))
        if checkpoints:
            latest = max(checkpoints, key=lambda p: p.stat().st_mtime)
            file_size = latest.stat().st_size / (1024 * 1024)  # MB
            mtime = latest.stat().st_mtime
            from datetime import datetime
            mtime_str = datetime.fromtimestamp(mtime).strftime("%Y-%m-%d %H:%M:%S")
            print(f"  最新チェックポイント: {latest.name}")
            print(f"  ファイルサイズ: {file_size:.2f} MB")
            print(f"  更新日時: {mtime_str}")
        else:
            print("  チェックポイントが見つかりません")
            print("  学習が開始されていないか、エラーで停止している可能性があります")
    
    # 6. ログファイルの確認
    log_files = list(exp_dir.glob("*.log")) + list(exp_dir.glob("train.log"))
    if log_files:
        log_file = log_files[0]
        print(f"\n✓ ログファイル: {log_file.name}")
        # ログの最後の数行を表示
        try:
            with open(log_file, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                if lines:
                    print(f"  ログの最後の5行:")
                    for line in lines[-5:]:
                        line = line.rstrip()
                        if line:
                            print(f"    {line}")
        except Exception as e:
            print(f"  ログ読み込みエラー: {e}")
    else:
        # run.shのログを探す
        run_log = recipe_dir / "train.log"
        if run_log.exists():
            print(f"\n✓ 学習ログ: {run_log}")
        else:
            print("\n⚠ ログファイルが見つかりません")

# 全データセット条件の学習状況を確認
for dataset_name, recipe_dir in recipe_dirs.items():
    check_training_status(recipe_dir, dataset_name)



【full100】学習状況チェック
✓ Recipeディレクトリ: /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1
❌ データ準備が未実行です（dump/24kディレクトリが見つかりません）
  → セル14で生成されたコマンドをターミナルで実行してください
  → Phase 4: データ準備（Stage 1-5）を参照
❌ Fine-tuning設定が未実行です（downloadsディレクトリが見つかりません）
  → セル16で生成されたコマンドをターミナルで実行してください
  → Phase 5: Fine-tuning設定を参照

❌ 学習が開始されていません（expディレクトリが存在しません）

📋 学習を開始する前に以下を確認:
  ⚠ 以下のステップが未完了です:
     - Phase 4: データ準備（Stage 1-5）
     - Phase 5: Fine-tuning設定

  → まず上記のステップを完了してから、Phase 6: 学習実行（Stage 6）を実行してください

【phone_min4】学習状況チェック
✓ Recipeディレクトリ: /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_phone_min4/tts1
❌ データ準備が未実行です（dump/24kディレクトリが見つかりません）
  → セル14で生成されたコマンドをターミナルで実行してください
  → Phase 4: データ準備（Stage 1-5）を参照
❌ Fine-tuning設定が未実行です（downloadsディレクトリが見つかりません）
  → セル16で生成されたコマンドをターミナルで実行してください
  → Phase 5: Fine-tuning設定を参照

❌ 学習が開始されていません（expディレクトリが存在しません）

📋 学習を開始する前に以下を確認:
  ⚠ 以下のステップが未完了です:
     - Phase 4: データ準備（Stage 1-5）
     - Phase 5: Fine-tuning設定

  → まず上記のステップを完了してから、Phase 

## 📋 解決策まとめ：学習を開始するための手順

### 現在の状況

学習ディレクトリが存在しない = **学習がまだ開始されていません**

これは正常です。ノートブックはコマンドを生成するだけで、実際の実行はターミナルで手動実行する必要があります。

### 学習を開始するための手順

#### ステップ1: データ準備（Phase 4）を実行 ⚠️ 未完了

ターミナルで以下のコマンドを実行（セル14を参照）:

```bash
cd /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1
source ../../../tools/venv/bin/activate
./run.sh --stage 1 --stop-stage 5 [その他のオプション...]
```

#### ステップ2: Fine-tuning設定（Phase 5）を実行 ⚠️ 未完了

ターミナルで以下のコマンドを実行（セル16を参照）:

```bash
cd /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1
source ../../../tools/venv/bin/activate
espnet_model_zoo_download --unpack true --cachedir downloads kan-bayashi/jsut_vits_accent_with_pause
```

#### ステップ3: 学習実行（Phase 6）を実行 ⚠️ 未完了

ターミナルで以下のコマンドを実行（セル18を参照）:

```bash
cd /mnt/c/dev/minimal-feature-corpus-tts/espnet/egs2/jvs002_full100/tts1
source ../../../tools/venv/bin/activate
nohup ./run.sh --stage 6 [その他のオプション...] > train.log 2>&1 &
```

#### ステップ4: 学習状況を再確認

学習を開始した後、このノートブックの**セル24を再実行**して、詳細な学習状況を確認してください。

### 重要なポイント

- ✅ **Recipeディレクトリは作成済み**（セル12で完了）
- ❌ **データ準備が未実行**（Phase 4が必要）
- ❌ **Fine-tuning設定が未実行**（Phase 5が必要）
- ❌ **学習が未開始**（Phase 6が必要）

各データセット条件（full100, phone_min4, feat_top10）ごとに、上記のステップを繰り返してください。


## トラブルシューティング

よくある問題と解決方法。


## ✅ 正常な状態です！

### 📊 現在の状況

**これはエラーではありません！** 学習の準備段階が正常に完了しています。

#### ✅ 完了していること

1. ✅ **Recipeディレクトリ作成完了**
   - full100, phone_min4, feat_top10 の3つのRecipeが作成済み

2. ✅ **データセット変換完了**
   - espnet_data/に3つのデータセットが準備済み

3. ✅ **コマンド生成完了**
   - Phase 4, 5, 6 のコマンドが生成済み

#### ⏳ これから実行すること（ターミナルで手動実行）

ノートブックは**コマンドを生成するだけ**です。実際の実行は**ターミナルで手動**で行う必要があります。

1. ⏳ **Phase 4: データ準備（Stage 1-5）**
   - セル14で生成されたコマンドをターミナルで実行

2. ⏳ **Phase 5: Fine-tuning設定**
   - セル16で生成されたコマンドをターミナルで実行

3. ⏳ **Phase 6: 学習実行（Stage 6）**
   - セル18で生成されたコマンドをターミナルで実行

### 🎯 次のアクション

まずは**full100データセット**で試してみましょう：

1. **ターミナルを開く**
2. **Phase 4のコマンドを実行**（セル14を参照）
3. **Phase 5のコマンドを実行**（セル16を参照）
4. **Phase 6のコマンドを実行**（セル18を参照）

各ステップが完了したら、**セル24を再実行**して進捗を確認してください。


### よくある問題

1. **エラー: Recipeが見つからない**
   - ESPNetのテンプレートディレクトリが存在するか確認
   - `espnet/egs2/TEMPLATE/tts1` が存在すること

2. **エラー: データセットパスが不正**
   - `db.sh` と `local/data.sh` のパスを確認
   - ESPNet形式データが正しく作成されているか確認

3. **エラー: トークンリストが見つからない**
   - Fine-tuning設定（Phase 5）を実行しているか確認
   - シンボリックリンクが正しく作成されているか確認

4. **学習が途中で止まる**
   - GPUメモリ不足の可能性
   - `train_config` のバッチサイズを小さくする
   - CPUで学習する場合、時間がかかります

5. **音声生成ができない**
   - モデルファイルが存在するか確認
   - ボコーダーファイルのパスを確認

### 参考リンク

- [ESPNet公式ドキュメント](https://espnet.github.io/espnet/)
- [ESPNet GitHub Issues](https://github.com/espnet/espnet/issues)
- [Qiita記事1](https://qiita.com/niwatiki/items/c2fca0307c8718bf3c61)
- [Qiita記事2](https://qiita.com/RRR_troisR/items/6288b9bdc6e725aa8440)
